<a href="https://colab.research.google.com/github/DHarley22/Prediction_case_malaria_mozambique/blob/1-xgboost-predictor-python-version/model_building_mozambique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
%%capture
!pip install gdown optuna


In [25]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np
import xgboost as xgb
import sklearn
import gdown
import optuna
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split_into_two
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from xgboost import XGBRegressor

In [26]:
%%capture

# The mozambique dataset is hosted here
# https://drive.google.com/file/d/1uHGOjxlxfDYY5E-aL9HVRGvheDa_wNyq/view?usp=sharing
dataset_drive_id = '1uHGOjxlxfDYY5E-aL9HVRGvheDa_wNyq'
dataset_output_file = 'downloaded_sheet.xlsx'

# The shape file relatd to the dataset is hosted here
# https://drive.google.com/drive/folders/14UJ7ZXWmNeL28sYAv6dObNsC42kQr4Ja?usp=sharing
shape_file_drive_id = '14UJ7ZXWmNeL28sYAv6dObNsC42kQr4Ja'
shape_output_folder = 'shape_files'

gdown.download_folder(id=shape_file_drive_id, output=shape_output_folder, quiet=False)
gdown.download(id=dataset_drive_id, output=dataset_output_file, quiet=False)


In [27]:
dataset = pd.read_csv(f"/content/{dataset_output_file}")
shape_file = gpd.read_file(f"/content/{shape_output_folder}/shape_file.shp")

In [28]:
# (Left) join the dataset to the shape_file using the "district" field
shaped_dataset = dataset.merge(
    shape_file[["district", "geometry"]],
    on="district",
    how="left"
)

# Convert the Dataframe into GeoDataframe and add some fields to it
shaped_dataset =  gpd.GeoDataFrame(shaped_dataset, geometry="geometry")

shaped_dataset = shaped_dataset.to_crs(epsg=3857)

# Calculate the area and centroid using the projected CRS
shaped_dataset["area"] = shaped_dataset.geometry.area
shaped_dataset["centroid_x"] = shaped_dataset.geometry.centroid.x
shaped_dataset["centroid_y"] = shaped_dataset.geometry.centroid.y

### Missing data handling

Rules:

- If {some condition} then {we do this}
- If {some condition} then {we do this}

In [29]:
# Handle missing data
# TODO:

### Test/Val/Test Split

Rules:

- We will use


In [30]:
TARGET_DISEASE_FIELD = "malaria_cases_u5"


# Relevant features
relevant_features = [
    # Environemental features
    'tmin', 'tmax', 'precipitation', 'ndvi','RH',
    # Socio-economic features
    'prop_poor', 'prop_Rural', 'prop_drinking_TreatedWater',
    # Spatial features
     'centroid_x', 'centroid_y'
]

# Test dataset predicate; use lines between March and June for testing
test_data_predicate = (shaped_dataset['year'] == 2018) & (shaped_dataset['month'].between(3, 6))

# Test dataset
X_test = shaped_dataset[test_data_predicate][relevant_features]
y_test = shaped_dataset[test_data_predicate][TARGET_DISEASE_FIELD]

# The rest of the dataset separated as X, y
X = shaped_dataset[~test_data_predicate][relevant_features]
y = shaped_dataset[~test_data_predicate][TARGET_DISEASE_FIELD]

print(X.shape, y.shape)
X_train, X_val, y_train, y_val = split_into_two(X, y, test_size=0.3, random_state=42)




(5088, 10) (5088,)


In [33]:

# Define the objective function for Optuna
def objective(trial):
    # Hyperparameter search space
    params = {
        'verbosity': 0,
        'booster': trial.suggest_categorical('booster', ['gbtree']),
        'objective': 'reg:squarederror',  # Regression objective
        'eval_metric': 'rmse',            # Root Mean Squared Error
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.2, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 0, 10),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 5000),
    }


    # Cross-validation
    cv_results = xgb.cv(
        params=params,
        dtrain=xgb.DMatrix(X_train, label=y_train),
        nfold=5,
        metrics='rmse',
        num_boost_round=params['n_estimators'],
        as_pandas=True,
        verbose_eval=50,
        early_stopping_rounds=20
    )

    return cv_results['test-rmse-mean'].iloc[-1]

# Run Optuna optimization (minization problem)
study = optuna.create_study(study_name="xgboost_model_study", direction="minimize")
study.optimize(objective, n_trials=100)

# Best hyperparameters
print("Best hyperparameters:", study.best_params)


[I 2025-01-28 16:35:26,789] A new study created in memory with name: xgboost_model_study


[0]	train-rmse:1777.35924+28.31030	test-rmse:1775.09794+119.79653
[50]	train-rmse:907.11931+16.45266	test-rmse:1020.29732+71.24673
[100]	train-rmse:734.90548+15.41758	test-rmse:919.21478+73.25653
[150]	train-rmse:653.80358+15.23004	test-rmse:890.49397+71.07101
[200]	train-rmse:592.19660+12.73526	test-rmse:877.38620+70.94232
[250]	train-rmse:542.68110+12.06193	test-rmse:869.84089+68.25990
[300]	train-rmse:502.08375+12.88430	test-rmse:864.84657+68.36436
[350]	train-rmse:467.23817+11.98358	test-rmse:861.18560+69.83962
[368]	train-rmse:455.95452+11.77670	test-rmse:861.10525+68.72035


[I 2025-01-28 16:35:35,956] Trial 0 finished with value: 860.8608398342842 and parameters: {'booster': 'gbtree', 'learning_rate': 0.04791104486254774, 'max_depth': 7, 'gamma': 2.1445268605181222e-08, 'subsample': 0.4353583788033487, 'reg_alpha': 0.00011375392653888646, 'reg_lambda': 3.510085627210054e-07, 'colsample_bytree': 0.6703317699475618, 'min_child_weight': 10, 'n_estimators': 2394}. Best is trial 0 with value: 860.8608398342842.


[0]	train-rmse:1822.72832+29.38229	test-rmse:1819.06748+123.15790
[50]	train-rmse:1798.62586+27.81125	test-rmse:1797.42077+121.68859
[100]	train-rmse:1774.60594+25.52622	test-rmse:1775.93523+120.57984
[150]	train-rmse:1755.04085+24.90063	test-rmse:1758.46962+119.19781
[200]	train-rmse:1733.36567+23.86543	test-rmse:1738.81031+117.42160
[250]	train-rmse:1715.46644+22.50012	test-rmse:1723.13466+116.65004
[300]	train-rmse:1696.65863+21.88108	test-rmse:1706.61122+115.34584
[350]	train-rmse:1676.27599+20.71145	test-rmse:1688.48640+114.12961
[400]	train-rmse:1654.48785+19.51058	test-rmse:1668.52626+112.41050
[450]	train-rmse:1637.46178+18.51875	test-rmse:1653.81886+111.47492
[500]	train-rmse:1617.99836+17.65189	test-rmse:1636.22267+110.03730
[550]	train-rmse:1602.13919+16.69193	test-rmse:1622.41595+108.99544
[600]	train-rmse:1585.13330+16.29581	test-rmse:1607.20080+107.44341
[650]	train-rmse:1568.17350+14.87409	test-rmse:1592.38058+106.90764
[700]	train-rmse:1552.76450+14.38827	test-rmse:1578

[I 2025-01-28 16:35:58,580] Trial 1 finished with value: 1160.0888877862149 and parameters: {'booster': 'gbtree', 'learning_rate': 0.0011515690933488261, 'max_depth': 5, 'gamma': 1.0651321910572778e-06, 'subsample': 0.2433512971234123, 'reg_alpha': 0.07617342423268673, 'reg_lambda': 9.365166733246727e-05, 'colsample_bytree': 0.278060317064422, 'min_child_weight': 7, 'n_estimators': 3434}. Best is trial 0 with value: 860.8608398342842.


...

[0]	train-rmse:1818.37770+29.32343	test-rmse:1814.86778+122.73613
[50]	train-rmse:1615.94393+22.39380	test-rmse:1634.21414+104.48891
[100]	train-rmse:1444.92646+18.70339	test-rmse:1481.68032+89.16045
[150]	train-rmse:1310.35866+16.78941	test-rmse:1368.29713+78.47677
[200]	train-rmse:1196.71401+15.29741	test-rmse:1276.28153+71.65312
[250]	train-rmse:1105.27266+14.38184	test-rmse:1206.85028+66.62631
[300]	train-rmse:1023.80348+13.75335	test-rmse:1146.08206+64.14263
[350]	train-rmse:954.76716+13.39293	test-rmse:1095.29835+62.54677
[400]	train-rmse:893.94311+13.08047	test-rmse:1052.21964+61.88015
[450]	train-rmse:843.40257+13.39618	test-rmse:1020.51582+61.32041
[500]	train-rmse:799.23785+13.49578	test-rmse:991.71042+61.39881
[550]	train-rmse:761.33121+13.84154	test-rmse:969.94320+61.25445
[600]	train-rmse:727.73457+13.80977	test-rmse:952.21301+61.29283
[650]	train-rmse:697.18380+13.61497	test-rmse:937.06752+61.42873
[700]	train-rmse:671.52670+13.63890	test-rmse:925.16432+61.74014
[750]	tra

[I 2025-01-28 17:55:09,908] Trial 98 finished with value: 850.798892326927 and parameters: {'booster': 'gbtree', 'learning_rate': 0.004055036456389516, 'max_depth': 10, 'gamma': 0.002007459350804415, 'subsample': 0.702287964776115, 'reg_alpha': 2.235454968993667e-07, 'reg_lambda': 0.5279218833635717, 'colsample_bytree': 0.469751878832235, 'min_child_weight': 7, 'n_estimators': 3430}. Best is trial 88 with value: 823.939288240677.


[0]	train-rmse:1819.42608+29.24845	test-rmse:1815.94282+122.93704
[50]	train-rmse:1631.91451+23.49806	test-rmse:1647.47816+107.41180
[100]	train-rmse:1472.72499+19.56530	test-rmse:1506.69297+94.62484
[150]	train-rmse:1338.00624+17.11444	test-rmse:1391.05492+84.95759
[200]	train-rmse:1222.64788+15.24362	test-rmse:1294.57767+78.54952
[250]	train-rmse:1127.28934+13.69476	test-rmse:1218.36197+74.62693
[300]	train-rmse:1043.25303+13.41855	test-rmse:1153.38795+72.37669
[350]	train-rmse:971.98091+12.91367	test-rmse:1099.36607+71.30992
[400]	train-rmse:909.69870+12.51565	test-rmse:1054.50913+70.94366
[450]	train-rmse:855.93704+12.79865	test-rmse:1017.24716+70.94947
[500]	train-rmse:809.86638+12.55421	test-rmse:987.00878+72.30148
[550]	train-rmse:770.32605+12.16681	test-rmse:962.51241+72.62099
[600]	train-rmse:734.48999+12.68991	test-rmse:941.20454+72.59865
[650]	train-rmse:702.80790+12.61252	test-rmse:923.56614+73.01688
[700]	train-rmse:675.60661+12.77190	test-rmse:910.51547+73.25355
[750]	tra

[I 2025-01-28 17:56:24,449] Trial 99 finished with value: 828.6769339591692 and parameters: {'booster': 'gbtree', 'learning_rate': 0.003231974549321658, 'max_depth': 10, 'gamma': 0.006205134437723041, 'subsample': 0.6179305125490046, 'reg_alpha': 1.2427185646027823e-06, 'reg_lambda': 0.2865774729961961, 'colsample_bytree': 0.8486046984969484, 'min_child_weight': 6, 'n_estimators': 2799}. Best is trial 88 with value: 823.939288240677.


Best hyperparameters: {'booster': 'gbtree', 'learning_rate': 0.0035209961607260945, 'max_depth': 9, 'gamma': 0.009596716772915171, 'subsample': 0.709187029135732, 'reg_alpha': 1.6638871630185252e-07, 'reg_lambda': 0.1951894062259646, 'colsample_bytree': 0.816001373149942, 'min_child_weight': 7, 'n_estimators': 3041}


In [32]:
# Here we will use the best parameters to train the model
best_model = xgb.train(
    study.best_params,
    xgb.DMatrix(X_train, label=y_train),
    num_boost_round=study.best_params["n_estimators"],
    # evals=[(dtrain_malaria, 'train'), (dval_malaria, 'val')],
    verbose_eval=50
)


NameError: name 'dtrain_malaria' is not defined